In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor, Pool, CatBoostClassifier
import catboost
from sklearn import metrics
from sklearn.model_selection import train_test_split

SEED = 42

In [2]:
def compute_metric(df):
    df = df.set_index('dt') 
    tender_price = df['Цена на арматуру']
    decision = df['Объем']
    start_date = df.index.min()
    end_date = df.index.max()
    _results = []
    _active_weeks = 0
    for report_date in pd.date_range(start_date, end_date, freq='W-MON'):
        if _active_weeks == 0:  # Пришла пора нового тендера
            _fixed_price = tender_price.loc[report_date]
            _active_weeks = int(decision.loc[report_date])
        _results.append(_fixed_price)
        _active_weeks += -1
    cost = sum(_results)
    return cost

In [3]:
train = pd.read_excel('data/train.xlsx')

Достаем информацию из дополнительных данных

In [4]:
def add_info(df):
    sirie = pd.read_excel('data/Цены на сырье.xlsx')
    sirie = sirie[sirie.dt.isin(df.dt)].dropna(axis=1)
    sirie.columns = [f'сырье_{x}' if sirie.columns[x]!='dt' else 'dt' for x in range(len(sirie.columns))]
    df = df.merge(sirie, how='left', on='dt')

    # Индекс стоимости грузоперевозок
    # Данные предоставляются по средам, но чтобы их учитывать в тендерах
    # мы меняем их дату на следующий понедельник
    gruz = pd.read_excel('data/Грузоперевозки.xlsx')
    gruz = pd.concat((gruz, df[['dt']])).sort_values('dt')
    gruz['Индекс стоимости грузоперевозок'] = gruz['Индекс стоимости грузоперевозок'].shift(1)
    gruz = gruz[gruz.dt.isin(df.dt)]
    df = df.merge(gruz, how='left', on='dt')

    # Данные у нас предоставляется в последний день каждого месяца
    # Меняем дату на следующие понедельник
    # Т.е. в понедельник определенного месяца будут одни и те же 
    metall = pd.read_excel('data/Показатели рынка металла.xlsx')
    metall = pd.concat((metall, df[~df.dt.isin(metall.dt)][['dt']])).sort_values('dt')
    metall = metall.fillna(method='ffill')
    metall = metall[metall.dt.isin(df.dt)]
    metall.columns = [f'металл_{x}' if metall.columns[x] != 'dt' else 'dt' for x in range(len(metall.columns))]
    df = df.merge(metall, how='left', on='dt')
    
    # То же самое, что и с показателями ранка металла
    materials = pd.read_excel('data/Данные рынка стройматериалов.xlsx') # То же, пока что только по месяцам, надо по дням
    materials = pd.concat((materials, df[~df.dt.isin(materials.dt)][['dt']])).sort_values('dt')
    materials = materials.fillna(method='ffill')
    materials = materials[materials.dt.isin(df.dt)].dropna()
    materials.columns = [f'материалы_{x}' if materials.columns[x] != 'dt' else 'dt' for x in range(len(materials.columns))]
    df = df.merge(materials, how='left', on='dt')
    
    # То же самое
    toplivo = pd.read_excel('data/Топливо.xlsx')
    toplivo = pd.concat((toplivo, df[~df.dt.isin(toplivo.dt)][['dt']])).sort_values('dt')
    toplivo = toplivo.fillna(method='ffill')
    toplivo = toplivo[toplivo.dt.isin(df.dt)].dropna()
    toplivo.columns = [f'топливо_{x}' if toplivo.columns[x] != 'dt' else 'dt' for x in range(len(toplivo.columns))]
    df = df.merge(toplivo, how='left', on='dt')
    
    # И опять:)
    lme = pd.read_excel('data/Индекс LME.xlsx').rename({'дата':'dt', 'цена': 'индекс lme'}, axis=1).dropna()
    lme = pd.concat((lme, df[~df.dt.isin(lme.dt)][['dt']])).sort_values('dt')
    lme = lme.fillna(method='ffill')
    lme = lme[lme.dt.isin(df.dt)].dropna()
    df = df.merge(lme, how='left', on='dt')
    
    # Берем цену акции, считаем как изменилась цена за предыдущую неделю
    chmf_stock = pd.read_csv(
        'data/CHMF Акции.csv',
        parse_dates=['Date']
    )[['Date', 'Change %']].rename({
        'Date': 'dt',
        'Change %': 'chmf_change'
    }, axis=1)
    chmf_stock['chmf_change'] = chmf_stock['chmf_change'].str.replace('%', '').astype(np.float32) / 100 + 1
    # Собираем информацию по изменению цены за предыдущие 5 дней
    chmf_stock['chmf_change'] = chmf_stock.chmf_change *\
        chmf_stock.chmf_change.shift(-1) *\
        chmf_stock.chmf_change.shift(-2) *\
        chmf_stock.chmf_change.shift(-3) *\
        chmf_stock.chmf_change.shift(-4)
    # Берем данные за пятницу и меняем дату на следующий понедельник, когда будет проводиться тендер
    chmf_stock = chmf_stock[chmf_stock.dt.dt.day_of_week == 4].dropna()
    chmf_stock = pd.concat((chmf_stock, df[~df.dt.isin(chmf_stock.dt)][['dt']])).sort_values('dt')
    chmf_stock = chmf_stock.fillna(method='ffill')
    chmf_stock = chmf_stock[chmf_stock.dt.isin(df.dt)].dropna()
    df = df.merge(chmf_stock, how='left', on='dt')
    
    # То же самое
    magn_stock = pd.read_csv(
        'data/MAGN Акции.csv',
        parse_dates=['Дата']
    )[['Дата', 'Изм. %']].rename({
        'Дата': 'dt',
        'Изм. %': 'magn_change'
    }, axis=1)
    magn_stock['magn_change'] = magn_stock['magn_change'].str.replace('%', '').str.replace(',','.').astype(np.float32)/100+1
    magn_stock['magn_change'] = magn_stock.magn_change *\
        magn_stock.magn_change.shift(-1) *\
        magn_stock.magn_change.shift(-2) *\
        magn_stock.magn_change.shift(-3) *\
        magn_stock.magn_change.shift(-4)
    magn_stock = magn_stock[magn_stock.dt.dt.day_of_week == 4].dropna()
    magn_stock = pd.concat((magn_stock, df[~df.dt.isin(magn_stock.dt)][['dt']])).sort_values('dt')
    magn_stock = magn_stock.fillna(method='ffill')
    magn_stock = magn_stock[magn_stock.dt.isin(df.dt)].dropna()
    df = df.merge(magn_stock, how='left', on='dt')
    
    # То же самое:)
    nlmk_stock = pd.read_csv(
        'data/NLMK Акции.csv',
        parse_dates=['Date']
    )[['Date', 'Change %']].rename({
        'Date': 'dt',
        'Change %': 'nlmk_change'
    }, axis=1)
    nlmk_stock['nlmk_change'] = nlmk_stock['nlmk_change'].str.replace('%', '').str.replace(',','.').astype(np.float32)/100+1
    nlmk_stock['nlmk_change'] = nlmk_stock.nlmk_change *\
        nlmk_stock.nlmk_change.shift(-1) *\
        nlmk_stock.nlmk_change.shift(-2) *\
        nlmk_stock.nlmk_change.shift(-3) *\
        nlmk_stock.nlmk_change.shift(-4)
    nlmk_stock = nlmk_stock[nlmk_stock.dt.dt.day_of_week == 4].dropna()
    nlmk_stock = pd.concat((nlmk_stock, df[~df.dt.isin(nlmk_stock.dt)][['dt']])).sort_values('dt')
    nlmk_stock = nlmk_stock.fillna(method='ffill')
    nlmk_stock = nlmk_stock[nlmk_stock.dt.isin(df.dt)].dropna()
    df = df.merge(nlmk_stock, how='left', on='dt')


    df['Изменение цены на арматуру'] = df['Цена на арматуру']/df['Цена на арматуру'].shift(1)
    df['month'] = df.dt.dt.month
    df = df.dropna(axis=1, how='all')

    return df

In [5]:
train = add_info(train)

month_stats = dict(train.groupby('month')['Изменение цены на арматуру'].median())
train['Изменение в месяце'] = train['month'].apply(lambda x: month_stats[x])

/var/folders/bb/jzxxldrn52l0stzqmszjx6840000gn/T/ipykernel_48116/1501421170.py:72: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  magn_stock = pd.read_csv(


In [6]:
def make_features(df):
    """Создание фичей по имеющимся у нас данным

    Args:
        df : Данные, которые у нас имеются для рекомендации к тендеру

    Returns:
        list: Список из фичей, которые достали из данных
    """
    cur_values = []

    # Как в последние 18 недель изменялась цена на арматуру
    cur_change = df['Изменение цены на арматуру'][-18:].tolist()
    cur_values.append(cur_change)
    
    # Во сколько раз изменилась цена на арматуру за последние 18 недель
    a = 1
    for x in cur_change:
        a *= x
    cur_values.append([a])
    
    # Средняя цена на арматуру в последние 10 недель
    cur_values.append([
        np.mean(df['Цена на арматуру'][-10:])
    ])
    cur_values.append([df['Изменение в месяце'].tolist()[-1]])

    # Для каждой колонке из таблицы "Цены на сырье" вычисляем во сколько раз изменилась
    # цена за последние 1,2 месяца и за последнюю неделю
    cur_values.append([df[x].tolist()[-1]/df[x].tolist()[-10] if df[x].tolist()[-10]!=0 else np.nan
                       for x in df.columns if 'сырье' in x])
    cur_values.append([df[x].tolist()[-1]/df[x].tolist()[-5] if df[x].tolist()[-5]!=0 else np.nan
                       for x in df.columns if 'сырье' in x])
    cur_values.append([df[x].tolist()[-1]/df[x].tolist()[-2] if df[x].tolist()[-2]!=0 else np.nan
                       for x in df.columns if 'сырье' in x])
    # cur_values.append([np.nanmean([
    #     df[x].tolist()[-1]/df[x].tolist()[-2] if df[x].tolist()[-2]!=0 else np.nan for x in df.columns if 'сырье' in x
    # ])])
    
    # Вычисляем на сколько и во сколько раз изменился индекс стоимости грузоперевозок
    # за последнюю неделю и 1,2 месяца
    cur_values.append([df['Индекс стоимости грузоперевозок'].tolist()[-1]-\
                       df['Индекс стоимости грузоперевозок'].tolist()[-2],
                       df['Индекс стоимости грузоперевозок'].tolist()[-1]-\
                       df['Индекс стоимости грузоперевозок'].tolist()[-5],
                       df['Индекс стоимости грузоперевозок'].tolist()[-1]-\
                       df['Индекс стоимости грузоперевозок'].tolist()[-10],
                       df['Индекс стоимости грузоперевозок'].tolist()[-1]-\
                       df['Индекс стоимости грузоперевозок'].tolist()[-20],

                       df['Индекс стоимости грузоперевозок'].tolist()[-1]/\
                       df['Индекс стоимости грузоперевозок'].tolist()[-2],
                       df['Индекс стоимости грузоперевозок'].tolist()[-1]/\
                       df['Индекс стоимости грузоперевозок'].tolist()[-5],
                       df['Индекс стоимости грузоперевозок'].tolist()[-1]/\
                       df['Индекс стоимости грузоперевозок'].tolist()[-10],
                       df['Индекс стоимости грузоперевозок'].tolist()[-1]/\
                       df['Индекс стоимости грузоперевозок'].tolist()[-20],])

    # Для каждой колонке из таблицы "Показатели рынка металла" считаем во сколько раз изменилась цена
    # за последние 1,2,4 месяца и берем среднее значение
    cur_values.append([
        # За последний месяц
        np.nanmean([df[x].tolist()[-1]/df[x].tolist()[-5]\
                    if df[x].tolist()[-5]!=0 else np.nan\
                    for x in df.columns if 'металл' in x]),
        
        # За последние 2 месяца
        np.nanmean([df[x].tolist()[-1]/df[x].tolist()[-10]\
                    if df[x].tolist()[-10]!=0 else np.nan\
                    for x in df.columns if 'металл' in x]),
        
        # За последние 4 месяца
        np.nanmean([df[x].tolist()[-1]/df[x].tolist()[-20]\
                    if df[x].tolist()[-20]!=0 else np.nan\
                    for x in df.columns if 'металл' in x]),
    ])

    # Для каждой колонке из таблицы о стройматериалах высчитываем во склько раз изменилась
    # изменилась цена за последние 1,2,4 месяца
    cur_values.append([
        np.nanmean([df[x].tolist()[-1]/df[x].tolist()[-10]\
                    if df[x].tolist()[-10]!=0 else np.nan\
                    for x in df.columns if 'материалы' in x]),
        np.nanmean([df[x].tolist()[-1]/df[x].tolist()[-5]\
                    if df[x].tolist()[-5]!=0 else np.nan\
                    for x in df.columns if 'материалы' in x]),
        np.nanmean([df[x].tolist()[-1]/df[x].tolist()[-20]\
                    if df[x].tolist()[-20]!=0 else np.nan\
                    for x in df.columns if 'материалы' in x]),
    ])
    # То же самое, только для топлива
    cur_values.append([df[x].tolist()[-1]-df[x].tolist()[-10]\
                    if df[x].tolist()[-10]!=0 else np.nan\
                    for x in df.columns if 'топливо' in x])
    cur_values.append([df[x].tolist()[-1]-df[x].tolist()[-20]\
                    if df[x].tolist()[-20]!=0 else np.nan\
                    for x in df.columns if 'топливо' in x])
    
    # На сколько пунктов изменился индекс lme за последние 1,2,4 месяца
    cur_values.append([
        df['индекс lme'].tolist()[-1] - df['индекс lme'].tolist()[-5],
        df['индекс lme'].tolist()[-1] - df['индекс lme'].tolist()[-10],
        df['индекс lme'].tolist()[-1] - df['индекс lme'].tolist()[-20],
    ])
    
    # Как изменилась цена акций за последние 5 недель
    cur_values.append([
        np.nanprod(df['chmf_change'].tolist()[-5:]),
        np.nanprod(df['magn_change'].tolist()[-5:]),
        np.nanprod(df['nlmk_change'].tolist()[-5:]),
    ])

    return np.concatenate(cur_values)

In [7]:
# На какое количество недель можем предсказывать
SLICE = 9

X_data, y_data = [], []
for i in range(20, len(train)-10):
    cur_df = train.iloc[:i] # Данные, которые у нас есть
    X_data.append(make_features(cur_df))

    cur_targets = []
    next_price = train['Цена на арматуру'][i:i+SLICE].tolist() # Цены в следующие моменты времени
    for i in range(len(next_price)):
        # Если цена пойдет выше -> надо покупать, тогда ставим 1, иначе - 0
        cur_targets.append(int(next_price[i] > cur_df['Цена на арматуру'].tolist()[-1]))

    y_data.append(cur_targets)

# В первых 5 строках много пустых значений
X_data, y_data = np.array(X_data[5:]), np.array(y_data[5:])

/var/folders/bb/jzxxldrn52l0stzqmszjx6840000gn/T/ipykernel_48116/3438699329.py:64: RuntimeWarning: Mean of empty slice
  np.nanmean([df[x].tolist()[-1]/df[x].tolist()[-5]\
/var/folders/bb/jzxxldrn52l0stzqmszjx6840000gn/T/ipykernel_48116/3438699329.py:69: RuntimeWarning: Mean of empty slice
  np.nanmean([df[x].tolist()[-1]/df[x].tolist()[-10]\
/var/folders/bb/jzxxldrn52l0stzqmszjx6840000gn/T/ipykernel_48116/3438699329.py:74: RuntimeWarning: Mean of empty slice
  np.nanmean([df[x].tolist()[-1]/df[x].tolist()[-20]\
/var/folders/bb/jzxxldrn52l0stzqmszjx6840000gn/T/ipykernel_48116/3438699329.py:82: RuntimeWarning: Mean of empty slice
  np.nanmean([df[x].tolist()[-1]/df[x].tolist()[-10]\
/var/folders/bb/jzxxldrn52l0stzqmszjx6840000gn/T/ipykernel_48116/3438699329.py:85: RuntimeWarning: Mean of empty slice
  np.nanmean([df[x].tolist()[-1]/df[x].tolist()[-5]\
/var/folders/bb/jzxxldrn52l0stzqmszjx6840000gn/T/ipykernel_48116/3438699329.py:88: RuntimeWarning: Mean of empty slice
  np.nanmean([df[x

Подбор параметров

Кол-во следующих ноликов. В начале приходится примерно 40% единичек, чтобы модельки не предсказывала слишком много поставим threshold 0.5

In [8]:
np.mean(y_data, axis=0)

array([0.36438356, 0.43287671, 0.44657534, 0.45479452, 0.46849315,
       0.48219178, 0.49863014, 0.51232877, 0.52876712])

In [9]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,

    eval_metric='AUC',
    thread_count=6,
    random_seed=SEED,
    verbose=False
)
params = {
    'depth': [3,4,5],
    'l2_leaf_reg': [3,4,5],
    'boosting_type': ['Plain', 'Ordered'],
    'bootstrap_type': ['Bayesian', 'Bernoulli', 'MVS'],
    'random_strength': [1, 1.2, 1.4, 1.6]
}
model.grid_search(params, X_data, y_data[:, 0], plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 0.8225
bestIteration = 18

Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
0:	loss: 0.8225000	best: 0.8225000 (0)	total: 1.28s	remaining: 4m 34s

bestTest = 0.8108333333
bestIteration = 5

Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
1:	loss: 0.8108333	best: 0.8225000 (0)	total: 2.45s	remaining: 4m 21s

bestTest = 0.8025
bestIteration = 36

Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
2:	loss: 0.8025000	best: 0.8225000 (0)	total: 3.51s	remaining: 4m 9s

bestTest = 0.8233333333
bestIteration = 46

Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
3:	loss: 0.8233333	best: 0.8233333 (3)	total: 4.53s	remaining: 4m

bestTest = 0.80083333

{'params': {'depth': 4,
  'l2_leaf_reg': 4,
  'random_strength': 1.4,
  'boosting_type': 'Ordered',
  'bootstrap_type': 'Bernoulli'},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
            

In [10]:
best_params = model.get_params()
best_params

{'iterations': 1000,
 'learning_rate': 0.03,
 'thread_count': 6,
 'random_seed': 42,
 'verbose': False,
 'eval_metric': 'AUC',
 'depth': 4,
 'l2_leaf_reg': 4,
 'random_strength': 1.4,
 'boosting_type': 'Ordered',
 'bootstrap_type': 'Bernoulli'}

{'iterations': 1000,
 'learning_rate': 0.03,
 'thread_count': 6,
 'random_seed': 42,
 'verbose': False,
 'eval_metric': 'AUC',
 'depth': 4,
 'l2_leaf_reg': 4,
 'random_strength': 1.4,
 'boosting_type': 'Ordered',
 'bootstrap_type': 'Bernoulli'}

In [11]:
models = []
scores = []
threshold = []
for slice in range(SLICE):
    cur_target = y_data[:, slice]

    X_train, X_valid, y_train, y_valid = train_test_split(X_data, cur_target, test_size=0.15, random_state=SEED)

    model = CatBoostClassifier(**best_params)
    model.fit(X_train, y_train, eval_set=Pool(X_valid, y_valid))

    # Берем фичи, которые хотя бы немного важны
    importance = pd.DataFrame({
        'col': list(range(X_train.shape[1])),
        'imp': model.get_feature_importance()
    }).sort_values('imp')
    good_cols = importance[importance > 0.01].dropna().col.astype('int').to_numpy()

    model.fit(X_train[:, good_cols], y_train, eval_set=Pool(X_valid[:, good_cols], y_valid))
    
    preds = model.predict_proba(X_valid[:, good_cols])[:, 1]
    valid_score = metrics.roc_auc_score(y_valid, preds)
    scores.append(valid_score)

    print(f'Slice {slice} Score: {valid_score}')

    models.append((model, good_cols))

'Final score:', np.mean(scores)

Slice 0 Score: 0.8491847826086957
Slice 1 Score: 0.8928571428571429
Slice 2 Score: 0.92
Slice 3 Score: 0.9306666666666666
Slice 4 Score: 0.956
Slice 5 Score: 0.988095238095238
Slice 6 Score: 0.9522546419098143
Slice 7 Score: 0.9386666666666666
Slice 8 Score: 0.9206989247311829


('Final score:', 0.9276026737261562)

In [12]:
import pickle
pickle.dump([models, month_stats], open('models.pkl', 'wb'))

-----